In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [16]:
data_c = np.load('/fastio2/dasha/double_pulse/MCTree_Corsika.npy', mmap_mode = 'r')
data_n = np.load('/fastio2/dasha/double_pulse/MCTree_NuTau_HESECheck.npy', mmap_mode = 'r')

In [17]:
print(data_c.shape,data_n.shape)

(7156067, 1) (6374299, 1)


In [18]:
data_n.dtype

dtype([('id', [('run_id', '<u4'), ('sub_run_id', '<u4'), ('event_id', '<u4'), ('sub_event_id', '<u4')]), ('neutrino', [('tree_id', '<u4', (2,)), ('pdg', '<i4'), ('energy', '<f4'), ('position', '<f4', (3,)), ('direction', '<f4', (2,)), ('time', '<f4'), ('length', '<f4')]), ('nutau', [('tree_id', '<u4', (2,)), ('parent_id', '<u4', (2,)), ('pdg', '<i4'), ('parent_pdg', '<i4'), ('children_pdgs', '<i4', (10,)), ('energy', '<f4'), ('position', '<f4', (3,)), ('direction', '<f4', (2,)), ('time', '<f4'), ('length', '<f4')], (10,)), ('taus', [('tree_id', '<u4', (2,)), ('parent_id', '<u4', (2,)), ('pdg', '<i4'), ('parent_pdg', '<i4'), ('children_pdgs', '<i4', (10,)), ('energy', '<f4'), ('position', '<f4', (3,)), ('direction', '<f4', (2,)), ('time', '<f4'), ('length', '<f4')], (10,)), ('muons', [('tree_id', '<u4', (2,)), ('parent_id', '<u4', (2,)), ('pdg', '<i4'), ('parent_pdg', '<i4'), ('children_pdgs', '<i4', (10,)), ('energy', '<f4'), ('position', '<f4', (3,)), ('direction', '<f4', (2,)), ('tim

In [29]:
mask_c_simtrim = data_c['keys']['simtrimmer'] == True
mask_c_convent = data_c['keys']['conventional'] == True
mask_c_filtermask = data_c['keys']['filtermask'] == True
mask_c_rawdata = data_c['keys']['weights'] == True

mask_n_simtrim = data_n['keys']['simtrimmer'] == True
mask_n_convent = data_n['keys']['conventional'] == True
mask_n_filtermask = data_n['keys']['filtermask'] == True
mask_n_rawdata = data_n['keys']['weights'] == True

In [31]:
print(data_c['keys']['weights'])

[[False]
 [False]
 [ True]
 ...
 [False]
 [ True]
 [False]]


In [39]:
print("RAW DATA            Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_rawdata)/data_c.shape[0]*100,np.sum(mask_n_rawdata)/data_n.shape[0]*100))
print("SIM TRIMMER         Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_simtrim)/data_c.shape[0]*100,np.sum(mask_n_simtrim)/data_n.shape[0]*100))
print("PASSED CONVENTIONAL Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_convent)/data_c.shape[0]*100,np.sum(mask_n_convent)/data_n.shape[0]*100))
print("FILTERMASK          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_filtermask)/data_c.shape[0]*100,np.sum(mask_n_filtermask)/data_n.shape[0]*100))

RAW DATA            Corsika =     33.60%, Genie =     77.38%
SIM TRIMMER         Corsika =     33.11%, Genie =      0.00%
PASSED CONVENTIONAL Corsika =      4.53%, Genie =     28.97%
FILTERMASK          Corsika =    100.00%, Genie =    100.00%


def I3RawDataCleaner(frame):       
    if not (('PassedConventional' in frame and frame['PassedConventional'].value == True) or ('SimTrimmer' in frame and frame['SimTrimmer'].value == True)):            
        frame.Delete('InIceRawData')
        frame.Delete('IceTopRawData')


In [40]:
mask_c_dataclean = mask_c_simtrim | mask_c_convent
mask_n_dataclean = mask_n_simtrim | mask_n_convent

print("DataClean          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_dataclean)/data_c.shape[0]*100,np.sum(mask_n_dataclean)/data_n.shape[0]*100))

DataClean          Corsika =     33.60%, Genie =     28.97%


Data Clean = Raw Data in Corsika, In Genie SimTrimmer was probably deleted from the frame

In [43]:
#OUR CHARGE CUT
mask_c_q = (data_c['qtot'] > 1000) & (data_c['qst'] > 400)
mask_n_q = (data_n['qtot'] > 1000) & (data_n['qst'] > 400)
print("Charge Cut          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_q)/data_c.shape[0]*100,np.sum(mask_n_q)/data_n.shape[0]*100))

Charge Cut          Corsika =      2.50%, Genie =     29.22%


In [45]:
#OVERLAP of CHARGE CUT and RAW DATA
mask_c_q_rd = mask_c_q & mask_c_rawdata
mask_n_q_rd = mask_n_q & mask_n_rawdata
print("Charge and RawData          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_q_rd)/data_c.shape[0]*100,np.sum(mask_n_q_rd)/data_n.shape[0]*100))

Charge and RawData          Corsika =      2.50%, Genie =     29.22%


Seems like the events that pass the charge cut have raw data

In [46]:
#OVERLAP of CHARGE CUT and SIMTRIMMER
mask_c_q_st = mask_c_q & mask_c_simtrim
mask_n_q_st = mask_n_q & mask_n_simtrim
print("Charge and SimTrim          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_q_st)/data_c.shape[0]*100,np.sum(mask_n_q_st)/data_n.shape[0]*100))

Charge and SimTrim          Corsika =      2.50%, Genie =      0.00%


In [48]:
#OVERLAP of CHARGE CUT and CONVENTIONAL
mask_c_q_con = mask_c_q & mask_c_convent
mask_n_q_con = mask_n_q & mask_n_convent
print("Charge and Conventional          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_q_con)/data_c.shape[0]*100,np.sum(mask_n_q_con)/data_n.shape[0]*100))

Charge and Conventional          Corsika =      2.29%, Genie =     27.68%


What about all the other keys needed to make images?

In [51]:
mask_c_pass = data_c['keys']['passed'] == True
mask_n_pass = data_n['keys']['passed'] == True
print("Has all the keys          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_pass)/data_c.shape[0]*100,np.sum(mask_n_pass)/data_n.shape[0]*100))

Has all the keys          Corsika =     26.28%, Genie =     68.14%


In [52]:
#OVERLAP of CHARGE CUT and ALL THE KEYS
mask_c_q_k = mask_c_q & mask_c_pass
mask_n_q_k = mask_n_q & mask_n_pass
print("Charge and All the Keys          Corsika = {0:9.2f}%, Genie = {1:9.2f}%".format(np.sum(mask_c_q_k)/data_c.shape[0]*100,np.sum(mask_n_q_k)/data_n.shape[0]*100))

Charge and All the Keys          Corsika =      2.50%, Genie =     29.22%


In [ ]:
fig = plt.figure(figsize=(5, 5),facecolor ='w')
ax = fig.add_subplot(111)
plt.rcParams.update({'font.size': 16})
plt.ticklabel_format(axis='both', style='sci', scilimits=(-5,5))
ax.set_xscale('linear')
ax.set_yscale('log')
ax.set_title("InteractionTypes", fontsize = 16)
ax.text(0.01,0.89,'#NuECC '+str(len(data_ec)), transform=ax.transAxes, color = "black",fontsize=12)

ax.set_xlabel('Type', fontsize = 16)                                                              
ax.set_ylabel("#Events", fontsize = 16)                                                   
ax.hist(data_e['weight']['InteractionType'])  
plt.show()